In [ ]:
# Try to construct the CNN #
# ------------------------ #
# In general, xdata represents images while ydata represents labels. #

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
import cv2
import tensorflow.contrib.slim as slim
ops.reset_default_graph()

# Start a graph session
sess = tf.Session()

# Load data
train_dir = '.\\images\\doge.jpg'
test_dir = ''
train_x = np.asarray([cv2.imread(train_dir)])

### Relevant paramerters of this CNN model
input -> 100C3-MP2 -> 200C2-MP2 -> 300C2-MP2 -> 400C2-MP2 -> 500C2-MP2 -> 600C2-MP2 -> 700C2 -> output
The 7th convolutional layer is directly connected with the fully connected layer which leads the output of this CNN model.

In [ ]:
# Set model parameters
batch_size = 1
learning_rate = 0.005
test_size = 20
im_width = train_x[0].shape[0]
im_height = train_x[0].shape[1]
labels_size = 10
num_channels = 3
train_epochs = 500
keep_prob = 1

conv1_output = 100
conv2_output = 200
conv3_output = 300
conv4_output = 400
conv5_output = 500
conv6_output = 600
conv7_output = 700
conv_size = 2

max_pool_size1 = 2
max_pool_size2 = 2
max_pool_size3 = 2
max_pool_size4 = 2
max_pool_size5 = 2
max_pool_size6 = 2

fully_connected1_size = 100

# Declare model placeholders
x_input_shape = (batch_size, im_width, im_height, num_channels)
x_input = tf.placeholder(tf.float32, shape=x_input_shape)
y_label = tf.placeholder(tf.int32, shape=(batch_size))

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1, dtype=tf.float32)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.zeros(shape = shape, dtype=tf.float32)
    return tf.Variable(initial)

### Declare tensor variables

The issue should be empasized here is that, for convolutional layer, the shape of **_kernal_** has 4 dimensions which are 
```python
[filter_height, filter_width, in_channels, out_channels]
```

while the **_input tensor_** of convolutional layer is restricted by

```python
[batch, in_height, in_width, in_channels]
```
The input channel of current layer equals to the output channel of previous one. Following is the declaration of convolutional kernal.
```python
conv1_weight = weight_variable([3, 3, num_channels, conv1_output])
conv2_weight = weight_variable([conv_size, conv_size, conv1_output, conv2_output])
conv3_weight = weight_variable([conv_size, conv_size, conv2_output, conv3_output])
...
```

### Output dimension after each layer

```python
max_pool1: (10, 128, 128, 100)
max_pool2: (10, 64, 64, 200)
max_pool3: (10, 32, 32, 300)
max_pool4: (10, 16, 16, 400)
max_pool5: (10, 8, 8, 500)
max_pool6: (10, 4, 4, 600)
relu7: (10, 4, 4, 700)
```
From the results above we can see that the input size of fully connected layer after those convolutional layer will finally be `(10, 4, 4, 700)`. Note that, each time max pool layer applied, the dimension will reduce related the kernel size in pool layer.

### Parameters interpretation

- [tf.nn.conv2d](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)<br>
    This function seems to only execute convolution calculation which does not include the activation of neural units in networks as there is no parameter to specify activation function.
<br>
- [slim.conv2d](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/conv2d)<br>
    [*_Slim_*](https://github.com/tensorflow/tensorflow/tree/r1.2/tensorflow/contrib/slim) performs as a interface that directly call a function to construct a convolutional layer that include activation function.


The use of `slim.conv2d` will lead a much more clear graph structure in tensorboard.

In [1]:
# Initialize Model Operations
def my_CNN(input):
    # 1st layer: 100C3-MP2
    conv_1 = slim.conv2d(input, 100, [3, 3], 1, padding='SAME', scope='conv1',activation_fn=tf.nn.relu)
    max_pool1 = slim.max_pool2d(conv_1, [2, 2], [2, 2], padding='SAME')

    # 2nd layer: 200C2-MP2
    conv_2 = slim.conv2d(max_pool1, 200, [2, 2], 1, padding='SAME', scope='conv2',activation_fn=tf.nn.relu)
    max_pool2 = max_pool_1 = slim.max_pool2d(conv_2, [2, 2], [2, 2], padding='SAME')

    # 3rd layer: 300C2-MP2
    conv_3 = slim.conv2d(max_pool2, 300, [2, 2], 1, padding='SAME', scope='conv3',activation_fn=tf.nn.relu)
    max_pool3 = max_pool_1 = slim.max_pool2d(conv_3, [2, 2], [2, 2], padding='SAME')

    # 4th layer: 400C2-MP2
    conv_4 = slim.conv2d(max_pool3, 400, [2, 2], 1, padding='SAME', scope='conv4',activation_fn=tf.nn.relu)
    max_pool4 = max_pool_1 = slim.max_pool2d(conv_4, [2, 2], [2, 2], padding='SAME')
    
    # 5th layer: 500C2-MP2
    conv_5 = slim.conv2d(max_pool4, 500, [2, 2], 1, padding='SAME', scope='conv5',activation_fn=tf.nn.relu)
    max_pool5 = max_pool_1 = slim.max_pool2d(conv_5, [2, 2], [2, 2], padding='SAME')

    # 6th layer: 600C2-MP2
    conv_6 = slim.conv2d(max_pool5, 600, [2, 2], 1, padding='SAME', scope='conv6',activation_fn=tf.nn.relu)
    max_pool6 = max_pool_1 = slim.max_pool2d(conv_6, [2, 2], [2, 2], padding='SAME')

    # 7th layer: 700C2
    conv_7 = slim.conv2d(max_pool6, 700, [2, 2], 1, padding='SAME', scope='conv7',activation_fn=tf.nn.relu)

    # Flat the output from conv layers for next fully connected layers
    flatten = slim.flatten(conv_7)
    
    # 1st fully connected layer
    fc1 = slim.fully_connected(slim.dropout(flatten, keep_prob), 1024,
                                   activation_fn=tf.nn.tanh, scope='fc1')

    # 2nd fully connected layer
    model_output = slim.fully_connected(slim.dropout(fc1, keep_prob), labels_size,
                                   activation_fn=None, scope='fc2')

    return(model_output)

model_output = my_CNN(x_input)

# Declare Loss function (softmax cross entropy)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_output, labels=y_label))

# Creat a prediction function
prediction = tf.nn.softmax(model_output)

# Create an optimizer
my_optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = my_optimizer.minimize(loss)

# Calculate accuracy function
# In this function, batch_prediction is the ouput result from the CNN
# while labels are the real label stored in dataset which trains the model
def get_acc(logits, labels):
    batch_predictions = np.argmax(logits, axis=1)
    bingo = np.sum(np.equal(batch_predictions, labels))
    return(100. * bingo/batch_predictions.shape[0])

# Run the model
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    init = tf.global_variables_initializer()
    sess.run(init)
    summary = sess.run(prediction, {x_input: train_x})
    for i in range(10):
        print(summary[0][i])

writer.close()

0.0177173
0.0625412
0.0079451
0.0780047
0.380059
0.0393354
0.00784571
0.0130283
0.0228492
0.370675


## Notes

1. Still need further working on input data sets.
    - shape, lables
    - randmly feeding
    - batches
<br>
<br>
2. Training and testing